In [1]:
pip install pandas openpyxl tkinter

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [3]:
pip install pandas openpyxl tkinter

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [7]:
# 앞에서 부터 데이터 검색
df.head()

NameError: name 'df' is not defined

## 헤더누락 보완, EDA
##  2,4,7,14,10,11,12,19,22,25,27,41,42,52,58,62,63,121,132,133,169,143,101,160

In [16]:
import pandas as pd
import os

# 🔹 CSV 파일이 저장된 폴더 경로
# folder_path = 'D:/시설/일일실적_해지현황(21년)/전사해지2401~12월/전사다시250123'
# folder_path = 'D:/시설/일일실적_해지현황(21년)/전사해지2401~12월/전사다시250123'
# folder_path = 'D:/시설/일일실적_해지현황(21년)/전사유지2401~12월/강북강원만'
# D:/시설/일일실적_해지현황(21년)/전사해지2401~12월/전사다시250123
folder_path = 'D:/시설/일일실적_해지현황(21년)/25년해지/1월'
# folder_path = 'D:/시설/일일실적_해지현황(21년)/전사유지2401~12월/25년/1월'

# 🔹 병합된 데이터를 저장할 엑셀 파일 경로
output_file_path = os.path.join(folder_path, 'merged_filtered_data.xlsx')

# 🔹 청크 크기 설정 (100,000행 단위로 처리)
chunk_size = 100000


def get_headers_from_first_csv(folder_path):
    """폴더 내 첫 번째 CSV 파일의 헤더를 불러옵니다."""
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            print(f"헤더를 가져올 파일: {file_path}")
            try:
                data = pd.read_csv(file_path, encoding='cp949', nrows=0)
                headers = data.columns.tolist()
                # 중복된 헤더 제거
                unique_headers = list(dict.fromkeys(headers))  # 중복 제거
                print(f"중복 제거 후 헤더 목록: {unique_headers}")
                return unique_headers
            except Exception as e:
                print(f"헤더를 불러오는 중 오류 발생: {e}")
    print("CSV 파일이 없습니다.")
    return []


def select_columns_cli(headers):
    """명령줄에서 사용자로부터 선택할 열을 입력받습니다."""
    print("사용 가능한 열 목록:")
    for i, header in enumerate(headers):
        print(f"{i + 1}. {header}")
   
    selected_indices = input("선택할 열 번호를 쉼표로 구분하여 입력하세요 (예: 1,3,5): ")
    selected_indices = [int(x.strip()) - 1 for x in selected_indices.split(",")]

    selected_columns = [headers[i] for i in selected_indices if 0 <= i < len(headers)]
    print(f"선택된 열: {selected_columns}")
    return selected_columns


def merge_and_save_chunks_to_excel(folder_path, selected_columns, output_file_path):
    """청크 단위로 데이터를 처리하여 엑셀 파일에 저장합니다."""
    header_written = False  # 헤더를 처음 한 번만 쓰기 위한 플래그

    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        # 기본 시트를 생성해 데이터가 없을 때 오류 방지
        pd.DataFrame().to_excel(writer, index=False, sheet_name='Sheet1')

        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                print(f"파일 처리 중: {file_path}")

                try:
                    # 청크 단위로 데이터 처리
                    for chunk in pd.read_csv(
                        file_path,
                        encoding='cp949',
                        usecols=lambda col: col in selected_columns,
                        chunksize=chunk_size,
                        dtype=str,  # 모든 열을 문자열로 처리해 메모리 사용 감소
                        low_memory=True
                    ):
                        # 중복 제거
                        chunk.drop_duplicates(inplace=True)
                       
                        # 첫 번째 헤더는 한 번만 기록
                        chunk.to_excel(
                            writer,
                            index=False,
                            header=not header_written,
                            startrow=writer.sheets['Sheet1'].max_row if header_written else 0,
                            sheet_name='Sheet1'
                        )
                        header_written = True  # 헤더가 기록되었음을 표시
                        print(f"{filename} 청크 처리 및 저장 완료.")
                except Exception as e:
                    print(f"{filename} 파일 처리 중 오류 발생: {e}")


def eda_on_merged_file(output_file_path):
    """병합된 데이터에 대해 EDA를 실행합니다."""
    try:
        # 병합된 데이터 불러오기
        print(f"{output_file_path} 파일 로드 중...")
        data = pd.read_excel(output_file_path, engine='openpyxl')
        print("파일 로드 성공. 데이터 분석 시작.")

        # 데이터 요약
        print("\n--- 데이터 요약 ---")
        print(data.info())

        # 데이터의 통계 요약
        print("\n--- 기본 통계 ---")
        print(data.describe(include='all'))

        # 결측치 확인
        print("\n--- 결측치 확인 ---")
        print(data.isnull().sum())

        # 데이터 샘플 출력
        print("\n--- 데이터 샘플 ---")
        print(data.head())

    except Exception as e:
        print(f"EDA 실행 중 오류 발생: {e}")


def main():
    headers = get_headers_from_first_csv(folder_path)
    if headers:
        selected_headers = select_columns_cli(headers)  # 명령줄에서 열 선택
        if selected_headers:
            merge_and_save_chunks_to_excel(folder_path, selected_headers, output_file_path)
            print(f"모든 데이터를 {output_file_path}에 저장 완료.")
            eda_on_merged_file(output_file_path)  # EDA 실행
        else:
            print("선택한 열이 없습니다. 프로그램을 종료합니다.")
    else:
        print("헤더를 가져올 수 없습니다. 프로그램을 종료합니다.")


# 실행
if __name__ == "__main__":
    main()

헤더를 가져올 파일: D:/시설/일일실적_해지현황(21년)/25년해지/1월\G000_02995_00013.csv
중복 제거 후 헤더 목록: ['관리본부코드', '관리본부명', '관리지사코드', '관리지사명', '고객번호', '고객명', '계약번호', '계약자명', '서비스번호', '서비스(대)', '서비스(중)', '서비스(소)', '영상전환일', '상호', '고객구분', '사업용구분', '주민등록번호', '사업자번호', '계약상태(대)', '계약상태(중)', '서비스상태(대)', '서비스상태(중)', '접속전화번호', '설치우편번호', '설치주소', '견적월정료', '월정료', '할인율', '계약개월수', '월정료면제사유', '면제시작일', '면제종료일', '계약보증금', '보증금면제사유', '판매상품비', '견적설치공사비', '설치공사할부개월', '계약공사비', '면제공사비', '실징수액', '계약시작일', '계약종료일', '영업구분', '추천구분', '추천경로', '영업본부코드', '영업본부명', '영업지사코드', '영업지사명', '영업지점명', '영업자사번', '영업자명', '영업자연락처', '모집유형', '영업채널', '가망고객번호', '추천본부코드', '추천본부명', '추천지사코드', '추천지사명', '유통망대분류', '유통망중분류', '유통망소분류', '추천자사번', '추천자명', '추천자유형', '추천채널', '정보제공자부서', '정보제공자사번', '정보제공자명', '정보제공자연락처', '청구본부코드', '청구본부명', '청구지사코드', '청구지사명', '청구자명', '합산여부', '수납구분', '세금계산서발행여부', '계산서발행여부', '선후납구분', '청구번호', '청구전화번호', '청구우편번호', '청구주소', '청약일자', '청약취소일자', '공사희망일', '공사완료일', '준공완료일', '관제개통일', '서비스개시일', '서비스재개시일', '공사의뢰유형', '회선방식(대)', '회선방식(중)', 'MUX', '전용회선번호', 'MIN번호

선택할 열 번호를 쉼표로 구분하여 입력하세요 (예: 1,3,5):  2,4,7,14,10,11,12,19,22,25,27,41,42,52,58,62,63,121,132,133,169,143,101,160


선택된 열: ['관리본부명', '관리지사명', '계약번호', '상호', '서비스(대)', '서비스(중)', '서비스(소)', '계약상태(대)', '서비스상태(중)', '설치주소', '월정료', '계약시작일', '계약종료일', '영업자명', '추천본부명', '유통망중분류', '유통망소분류', '계약최초서비스게시일', '시설구분', '요금구분', '제외사유', '영업구역정보', '해지일자', '실적채널']
파일 처리 중: D:/시설/일일실적_해지현황(21년)/25년해지/1월\G000_02995_00013.csv
G000_02995_00013.csv 청크 처리 및 저장 완료.
모든 데이터를 D:/시설/일일실적_해지현황(21년)/25년해지/1월\merged_filtered_data.xlsx에 저장 완료.
D:/시설/일일실적_해지현황(21년)/25년해지/1월\merged_filtered_data.xlsx 파일 로드 중...
파일 로드 성공. 데이터 분석 시작.

--- 데이터 요약 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8121 entries, 0 to 8120
Data columns (total 24 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   관리본부명       8121 non-null   object 
 1   관리지사명       8121 non-null   object 
 2   계약번호        8121 non-null   int64  
 3   서비스(대)      8121 non-null   object 
 4   서비스(중)      8121 non-null   object 
 5   서비스(소)      8121 non-null   object 
 6   상호          8118 non-null   object 
 7   계약상태(대)     8121 non

In [11]:
import pandas as pd

# 엑셀 파일 경로
output_file_path = 'D:/시설/일일실적_해지현황(21년)/전사유지2401~12월/강북강원만/merged_filtered_data - 복사본.xlsx'

def update_column_values(file_path, column_name, replacements):
    """엑셀 파일의 특정 열 값을 변경합니다."""
    try:
        # 엑셀 파일 읽기
        print(f"{file_path} 파일 로드 중...")
        data = pd.read_excel(file_path, engine='openpyxl')
        print("파일 로드 성공. 값 변경 작업 시작.")

        # 값 변경
        if column_name in data.columns:
            data[column_name] = data[column_name].replace(replacements)
            print(f"{column_name} 열의 값 변경 완료.")
        else:
            print(f"{column_name} 열이 데이터프레임에 없습니다.")

        # 변경된 데이터를 동일한 파일에 저장
        data.to_excel(file_path, index=False, engine='openpyxl')
        print(f"변경된 데이터를 {file_path}에 저장 완료.")
    except Exception as e:
        print(f"값 변경 작업 중 오류 발생: {e}")


# 변경할 값 설정
replacements = {
    "강원본부": "강북/강원본부",
    "서부본부": "강남/서부본부"
}

# 실행
update_column_values(output_file_path, "관리본부명", replacements)


D:/시설/일일실적_해지현황(21년)/전사유지2401~12월/강북강원만/merged_filtered_data - 복사본.xlsx 파일 로드 중...


KeyboardInterrupt: 

In [3]:
import pandas as pd
import os

# 🔹 CSV 파일이 저장된 폴더 경로
folder_path = 'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용'

# 🔹 병합된 데이터를 저장할 엑셀 파일 경로
output_file_path = os.path.join(folder_path, '강북유지2411월 eda포함_data.xlsx')

# 🔹 청크 크기 설정 (100,000행 단위로 처리)
chunk_size = 100000


def get_headers_from_first_csv(folder_path):
    """폴더 내 첫 번째 CSV 파일의 헤더를 불러옵니다."""
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            print(f"헤더를 가져올 파일: {file_path}")
            try:
                data = pd.read_csv(file_path, encoding='cp949', nrows=0)
                headers = data.columns.tolist()
                # 중복된 헤더 제거
                unique_headers = list(dict.fromkeys(headers))  # 중복 제거
                print(f"중복 제거 후 헤더 목록: {unique_headers}")
                return unique_headers
            except Exception as e:
                print(f"헤더를 불러오는 중 오류 발생: {e}")
    print("CSV 파일이 없습니다.")
    return []


def select_columns_cli(headers):
    """명령줄에서 사용자로부터 선택할 열을 입력받습니다."""
    print("사용 가능한 열 목록:")
    for i, header in enumerate(headers):
        print(f"{i + 1}. {header}")
   
    selected_indices = input("선택할 열 번호를 쉼표로 구분하여 입력하세요 (예: 1,3,5): ")
    selected_indices = [int(x.strip()) - 1 for x in selected_indices.split(",")]

    selected_columns = [headers[i] for i in selected_indices if 0 <= i < len(headers)]
    print(f"선택된 열: {selected_columns}")
    return selected_columns


def merge_and_save_chunks_to_excel(folder_path, selected_columns, output_file_path):
    """청크 단위로 데이터를 처리하여 엑셀 파일에 저장합니다."""
    header_written = False  # 헤더를 처음 한 번만 쓰기 위한 플래그

    with pd.ExcelWriter(output_file_path, engine='openpyxl') as writer:
        # 기본 시트를 생성해 데이터가 없을 때 오류 방지
        pd.DataFrame().to_excel(writer, index=False, sheet_name='Sheet1')

        for filename in os.listdir(folder_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(folder_path, filename)
                print(f"파일 처리 중: {file_path}")

                try:
                    # 청크 단위로 데이터 처리
                    for chunk in pd.read_csv(
                        file_path,
                        encoding='cp949',
                        usecols=lambda col: col in selected_columns,
                        chunksize=chunk_size,
                        dtype=str,  # 모든 열을 문자열로 처리해 메모리 사용 감소
                        low_memory=True
                    ):
                        # 중복 제거
                        chunk.drop_duplicates(inplace=True)
                       
                        # 첫 번째 헤더는 한 번만 기록
                        chunk.to_excel(
                            writer,
                            index=False,
                            header=not header_written,
                            startrow=writer.sheets['Sheet1'].max_row if header_written else 0,
                            sheet_name='Sheet1'
                        )
                        header_written = True  # 헤더가 기록되었음을 표시
                        print(f"{filename} 청크 처리 및 저장 완료.")
                except Exception as e:
                    print(f"{filename} 파일 처리 중 오류 발생: {e}")


def eda_on_merged_file(output_file_path):
    """병합된 데이터에 대해 EDA를 실행합니다."""
    try:
        # 병합된 데이터 불러오기
        print(f"{output_file_path} 파일 로드 중...")
        data = pd.read_excel(output_file_path, engine='openpyxl')
        print("파일 로드 성공. 데이터 분석 시작.")

        # 데이터 요약
        print("\n--- 데이터 요약 ---")
        print(data.info())

        # 데이터의 통계 요약
        print("\n--- 기본 통계 ---")
        print(data.describe(include='all'))

        # 결측치 확인
        print("\n--- 결측치 확인 ---")
        print(data.isnull().sum())

        # 데이터 샘플 출력
        print("\n--- 데이터 샘플 ---")
        print(data.head())

    except Exception as e:
        print(f"EDA 실행 중 오류 발생: {e}")


def main():
    headers = get_headers_from_first_csv(folder_path)
    if headers:
        selected_headers = select_columns_cli(headers)  # 명령줄에서 열 선택
        if selected_headers:
            merge_and_save_chunks_to_excel(folder_path, selected_headers, output_file_path)
            print(f"모든 데이터를 {output_file_path}에 저장 완료.")
            eda_on_merged_file(output_file_path)  # EDA 실행
        else:
            print("선택한 열이 없습니다. 프로그램을 종료합니다.")
    else:
        print("헤더를 가져올 수 없습니다. 프로그램을 종료합니다.")


# 실행
if __name__ == "__main__":
    main()

헤더를 가져올 파일: D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용\G009_03313_00079.csv
헤더를 불러오는 중 오류 발생: 'cp949' codec can't decode byte 0xab in position 12: illegal multibyte sequence
CSV 파일이 없습니다.
헤더를 가져올 수 없습니다. 프로그램을 종료합니다.


In [14]:
pip install pandas openpyxl pandas-profiling

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/262.6 kB ? eta -:--:--
   ----------------------------------- ---- 235.5/262.6 kB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 262.6/262.6 kB 5.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
   -------- ------------------------------- 1.0/4.7 MB 21.1 MB/s eta 0:00:01
   ---------------- ----------------------- 2.0/4.7 MB 25.1 MB/s eta 0:00:01
   ------------------------ --------------- 2.9/4.7 MB 26.8 MB/s eta 0:00:01
   -------------------------------- ------- 3.9/4.7 MB 24.8 MB/s eta 0:00:01
   ---------------------------------------  4.7/4.7 MB 23.3 MB/s eta 0:00:01
   ---------------------------------------- 4.7/4.7 MB 21.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/102.4 kB ?

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.4.2 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.


In [16]:
import pandas as pd
from pandas_profiling import ProfileReport

# 🔹 엑셀 파일 경로 (윈도우 경로는 역슬래시 \ 를 사용하거나 슬래시 / 를 사용합니다)
file_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월 eda포함_data.xlsx'

# 🔹 불러올 파일의 EDA 결과 HTML 파일 경로
eda_report_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월_eda_report.html'


def load_excel_file(file_path):
    """엑셀 파일 불러오기"""
    try:
        print(f"파일 {file_path} 로드 중...")
        data = pd.read_excel(file_path, engine='openpyxl')
        print(f"파일 로드 완료. 데이터의 크기: {data.shape}")
        return data
    except Exception as e:
        print(f"파일을 로드하는 중 오류 발생: {e}")
        return None


def perform_eda(data, report_path):
    """EDA 분석 및 HTML 보고서 생성"""
    try:
        print("EDA 분석 시작...")
       
        # pandas-profiling을 사용하여 EDA 보고서 생성
        profile = ProfileReport(
            data,
            title="강북유지2411월 데이터 EDA 보고서",
            explorative=True
        )
       
        # HTML 보고서 저장
        profile.to_file(report_path)
        print(f"EDA 보고서가 다음 경로에 저장되었습니다: {report_path}")
    except Exception as e:
        print(f"EDA 분석 중 오류 발생: {e}")


def main():
    """메인 실행 함수"""
    # 1️⃣ 엑셀 파일 불러오기
    data = load_excel_file(file_path)
   
    if data is not None:
        # 2️⃣ 데이터의 기본 정보 출력
        print("\n--- 데이터 정보 ---")
        print(data.info())
       
        # 3️⃣ 데이터의 결측치 개수 확인
        print("\n--- 결측치 확인 ---")
        print(data.isnull().sum())
       
        # 4️⃣ 데이터의 기본 통계 확인
        print("\n--- 기본 통계 ---")
        print(data.describe(include='all'))
       
        # 5️⃣ 데이터 샘플 미리보기
        print("\n--- 데이터 샘플 (상위 5건) ---")
        print(data.head())
       
        # 6️⃣ EDA 시각화 보고서 생성
        perform_eda(data, eda_report_path)


# 실행
if __name__ == "__main__":
    main()

AttributeError: module 'numba' has no attribute 'generated_jit'

In [18]:
pip install numba==0.55.2

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



ERROR: Ignored the following versions that require a different python version: 0.52.0 Requires-Python >=3.6,<3.9; 0.52.0rc3 Requires-Python >=3.6,<3.9; 0.53.0 Requires-Python >=3.6,<3.10; 0.53.0rc1.post1 Requires-Python >=3.6,<3.10; 0.53.0rc2 Requires-Python >=3.6,<3.10; 0.53.0rc3 Requires-Python >=3.6,<3.10; 0.53.1 Requires-Python >=3.6,<3.10; 0.54.0 Requires-Python >=3.7,<3.10; 0.54.0rc2 Requires-Python >=3.7,<3.10; 0.54.0rc3 Requires-Python >=3.7,<3.10; 0.54.1 Requires-Python >=3.7,<3.10; 0.55.0 Requires-Python >=3.7,<3.11; 0.55.0rc1 Requires-Python >=3.7,<3.11; 0.55.1 Requires-Python >=3.7,<3.11; 0.55.2 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement numba==0.55.2 (from versions: 0.1, 0.2, 0.3, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.8.0, 0.8.1, 0.9.0, 0.10.0, 0.10.1, 0.11.0, 0.12.0, 0.12.1, 0.12.2, 0.13.0, 0.13.2, 0.13.3, 0.13.4, 0.14.0, 0.15.1, 0.16.0, 0.17.0, 0.18.1, 0.18.2, 0.19.1, 0.19.2, 0.20.0, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.23.1, 0

In [20]:
pip install --upgrade pandas-profiling

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install --upgrade visions

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/104.8 kB ? eta -:--:--
   --------------- ------------------------ 41.0/104.8 kB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 104.8/104.8 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: visions
    Found existing installation: visions 0.7.4
    Uninstalling visions-0.7.4:
      Successfully uninstalled visions-0.7.4
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires visions[type_image_path]==0.7.4, but you have visions 0.7.6 which is incompatible.


In [32]:
import pandas as pd
from pandas_profiling import ProfileReport

# 🔹 엑셀 파일 경로 (윈도우 경로는 역슬래시 \ 를 사용하거나 슬래시 / 를 사용합니다)
file_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월 eda포함_data.xlsx'

# 🔹 불러올 파일의 EDA 결과 HTML 파일 경로
eda_report_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월_eda_report.html'

def load_excel_file(file_path):
    """엑셀 파일 불러오기"""
    try:
        print(f"파일 {file_path} 로드 중...")
        data = pd.read_excel(file_path, engine='openpyxl')
        print(f"파일 로드 완료. 데이터의 크기: {data.shape}")
        return data
    except Exception as e:
        print(f"파일을 로드하는 중 오류 발생: {e}")
        return None

def perform_eda(data, report_path):
    """EDA 분석 및 HTML 보고서 생성"""
    try:
        print("EDA 분석 시작...")
       
        # pandas-profiling을 사용하여 EDA 보고서 생성
        profile = ProfileReport(
            data,
            title="강북유지2411월 데이터 EDA 보고서",
            explorative=True
        )
       
        # HTML 보고서 저장
        profile.to_file(report_path)
        print(f"EDA 보고서가 다음 경로에 저장되었습니다: {report_path}")
    except Exception as e:
        print(f"EDA 분석 중 오류 발생: {e}")

def main():
    """메인 실행 함수"""
    # 1️⃣ 엑셀 파일 불러오기
    data = load_excel_file(file_path)
   
    if data is not None:
        # 2️⃣ 데이터의 기본 정보 출력
        print("\n--- 데이터 정보 ---")
        print(data.info())
       
        # 3️⃣ 데이터의 결측치 개수 확인
        print("\n--- 결측치 확인 ---")
        print(data.isnull().sum())
       
        # 4️⃣ 데이터의 기본 통계 확인
        print("\n--- 기본 통계 ---")
        print(data.describe(include='all'))
       
        # 5️⃣ 데이터 샘플 미리보기
        print("\n--- 데이터 샘플 (상위 5건) ---")
        print(data.head())
       
        # 6️⃣ EDA 시각화 보고서 생성
        perform_eda(data, eda_report_path)

# 실행
if __name__ == "__main__":
    main()

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.5/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.5/u/import-error

In [36]:
import sys
import subprocess
import importlib

# 🔹 필수 라이브러리 목록
required_libraries = ['pandas', 'openpyxl', 'pandas-profiling']

def install_and_import_packages(libraries):
    """필요한 라이브러리를 설치하고 가져옵니다."""
    for library in libraries:
        try:
            importlib.import_module(library)
            print(f"✅ {library} 라이브러리가 이미 설치되어 있습니다.")
        except ImportError:
            print(f"📦 {library} 라이브러리를 설치 중...")
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', library])
            print(f"✅ {library} 설치 완료.")
           
# 🔹 필수 라이브러리 설치 및 가져오기
install_and_import_packages(required_libraries)

import pandas as pd
from pandas_profiling import ProfileReport

# 🔹 엑셀 파일 경로 (윈도우 경로는 역슬래시 \ 를 사용하거나 슬래시 / 를 사용합니다)
file_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월 eda포함_data.xlsx'

# 🔹 불러올 파일의 EDA 결과 HTML 파일 경로
eda_report_path = r'D:/시설/일일실적_해지현황(21년)/강북유지2411월 검증용/강북유지2411월_eda_report.html'


def load_excel_file(file_path):
    """엑셀 파일 불러오기"""
    try:
        print(f"📂 파일 {file_path} 로드 중...")
        data = pd.read_excel(file_path, engine='openpyxl')
        print(f"✅ 파일 로드 완료. 데이터의 크기: {data.shape}")
        return data
    except Exception as e:
        print(f"❌ 파일을 로드하는 중 오류 발생: {e}")
        return None


def perform_eda(data, report_path):
    """EDA 분석 및 HTML 보고서 생성"""
    try:
        print("📊 EDA 분석 시작...")
       
        # pandas-profiling을 사용하여 EDA 보고서 생성
        profile = ProfileReport(
            data,
            title="강북유지2411월 데이터 EDA 보고서",
            explorative=True
        )
       
        # HTML 보고서 저장
        profile.to_file(report_path)
        print(f"✅ EDA 보고서가 다음 경로에 저장되었습니다: {report_path}")
    except Exception as e:
        print(f"❌ EDA 분석 중 오류 발생: {e}")


def main():
    """메인 실행 함수"""
    # 1️⃣ 엑셀 파일 불러오기
    data = load_excel_file(file_path)
   
    if data is not None:
        # 2️⃣ 데이터의 기본 정보 출력
        print("\n--- 데이터 정보 ---")
        print(data.info())
       
        # 3️⃣ 데이터의 결측치 개수 확인
        print("\n--- 결측치 확인 ---")
        print(data.isnull().sum())
       
        # 4️⃣ 데이터의 기본 통계 확인
        print("\n--- 기본 통계 ---")
        print(data.describe(include='all'))
       
        # 5️⃣ 데이터 샘플 미리보기
        print("\n--- 데이터 샘플 (상위 5건) ---")
        print(data.head())
       
        # 6️⃣ EDA 시각화 보고서 생성
        perform_eda(data, eda_report_path)


# 실행
if __name__ == "__main__":
    main()


✅ pandas 라이브러리가 이미 설치되어 있습니다.
✅ openpyxl 라이브러리가 이미 설치되어 있습니다.
📦 pandas-profiling 라이브러리를 설치 중...
✅ pandas-profiling 설치 완료.


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.5/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.5/u/import-error

In [ ]:
주어진 코드에서 위젯이 제대로 작동하지 않는 문제를 겪고 있는 것으로 보입니다. 이를 해결하기 위해 아래 내용을 확인해 보세요.

### 1. **ipywidgets 설치 및 활성화**
   - 코드를 실행하는 환경에서 `ipywidgets`가 설치되어 있는지 확인하세요. 설치되어 있지 않다면 다음 명령어로 설치하세요:
     ```bash
     pip install ipywidgets
     ```
   - Jupyter Notebook 또는 Jupyter Lab에서 위젯이 작동하도록 활성화하려면 다음 명령어를 실행하세요:
     ```bash
     jupyter nbextension enable --py widgetsnbextension
     jupyter nbextension install --py widgetsnbextension
     ```

### 2. **위젯 라이브러리 충돌 해결**
   - `Error displaying widget` 메시지가 나타나는 이유는 `ipywidgets`가 Jupyter 환경과 호환되지 않는 경우일 수 있습니다.
   - Jupyter Lab 사용 시 `jupyterlab_widgets`를 설치하고 환경을 재시작해보세요:
     ```bash
     pip install jupyterlab_widgets
     ```

### 3. **환경 문제 해결**
   - Jupyter Notebook 대신 Visual Studio Code 또는 Google Colab을 사용하면 위젯 관련 문제가 해결될 수 있습니다.
   - Colab에서는 아래 코드를 추가로 실행해야 합니다:
     ```python
     !pip install ipywidgets
     from google.colab import output
     output.enable_custom_widget_manager()
     ```

### 4. **코드 수정**
   - 체크박스와 위젯 생성 코드가 복잡하게 작성되어 충돌 가능성이 있습니다. 아래 간소화된 코드로 테스트해 보세요:

```python
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# CSV 파일 경로 설정
file_path = 'D:/시설/유지/24년11월유지가입자.cs'

# 데이터 읽기
try:
    df = pd.read_csv(file_path, encoding='cp949', dtype=str)
    df.fillna('', inplace=True)
except Exception as e:
    print(f"Error reading file: {e}")

# 입력 필드 생성
input_field = widgets.Text(placeholder="검색어 입력", description="검색:")

# 필터링 결과 출력 영역
output_area = widgets.Output()

# 필터 버튼 생성
filter_button = widgets.Button(description="필터 적용", button_style="success")

def apply_filter(b):
    with output_area:
        clear_output()
        try:
            if input_field.value.strip():
                filtered_df = df[df.apply(lambda row: row.astype(str).str.contains(input_field.value, case=False).any(), axis=1)]
                if not filtered_df.empty:
                    display(filtered_df.head())
                else:
                    print("조회된 데이터가 없습니다.")
            else:
                print("검색어를 입력하세요.")
        except Exception as e:
            print(f"Error: {e}")

filter_button.on_click(apply_filter)

# UI 표시
display(widgets.VBox([input_field, filter_button, output_area]))
```

### 5. **UI 디버깅**
   - 간소화된 코드로 UI가 정상 작동하는지 확인하세요. 문제가 없으면 점차 기능을 추가하며 테스트해 보세요.

문제를 해결해보시고, 추가적인 오류가 있으면 알려주세요!



In [ ]:
윈도우에서 Jupyter Notebook을 사용하는 경우 `Error displaying widget` 문제가 지속된다면, 아래의 단계를 따라 해결을 시도해 보세요.

---

## 1. **Jupyter Notebook 위젯 확장 설치 및 활성화**

Jupyter Notebook에서 위젯이 작동하려면 확장이 제대로 설정되어야 합니다. 아래 명령어를 실행하세요:

```bash
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
jupyter nbextension install --py widgetsnbextension
jupyter notebook
```

설치 후, Jupyter Notebook을 재시작하세요.

---

## 2. **환경 확인**
### **2.1 라이브러리 버전 확인**
아래 명령어로 `ipywidgets`와 `Jupyter Notebook`의 버전을 확인하세요:

```bash
pip show ipywidgets notebook
```

- 권장 버전:
  - **ipywidgets**: 7.x 이상 (최신 안정 버전)
  - **notebook**: 6.x 이상

### **2.2 최신 버전으로 업데이트**
버전이 낮거나 설치가 안 되어 있다면 다음 명령어로 업데이트하세요:

```bash
pip install --upgrade ipywidgets notebook
```

---

## 3. **대체로 Jupyter Lab 사용**
만약 `Jupyter Notebook`에서 위젯이 제대로 작동하지 않으면 `Jupyter Lab`을 사용하는 것도 좋은 대안입니다.

### **3.1 설치**
```bash
pip install jupyterlab
pip install jupyterlab_widgets
```

### **3.2 실행**
```bash
jupyter lab
```

---

## 4. **Google Chrome 사용**
Jupyter Notebook을 실행하는 브라우저를 Google Chrome으로 변경해보세요. 특정 브라우저에서는 위젯이 제대로 표시되지 않는 경우가 있습니다.

- 브라우저 변경 방법:
  - Jupyter Notebook 실행 시 명령어에 `--browser` 옵션 추가:
    ```bash
    jupyter notebook --browser=chrome
    ```

---

## 5. **Streamlit 또는 Dash로 전환 (대안)**
Jupyter Notebook에서 ipywidgets가 제대로 작동하지 않는 경우, 다른 UI 라이브러리를 사용하여 코드를 실행할 수도 있습니다.

### **Streamlit 설치**
```bash
pip install streamlit
```

### **Streamlit 실행 코드 예제**
```python
import pandas as pd
import streamlit as st

# 파일 경로
file_path = 'D:/시설/유지/24년11월유지가입자.cs'

# 데이터 읽기
try:
    df = pd.read_csv(file_path, encoding='cp949', dtype=str)
    df.fillna('', inplace=True)
except Exception as e:
    st.error(f"Error reading file: {e}")
    st.stop()

# 검색어 입력
query = st.text_input("검색어를 입력하세요")

# 데이터 필터링
if query:
    filtered_df = df[df.apply(lambda row: row.astype(str).str.contains(query, case=False).any(), axis=1)]
    if not filtered_df.empty:
        st.dataframe(filtered_df)
    else:
        st.warning("조회된 데이터가 없습니다.")
```

### 실행
```bash
streamlit run script_name.py
```

---

## 6. **Jupyter Notebook 재설치**
환경이 손상된 경우, Jupyter Notebook을 재설치하면 문제가 해결될 수 있습니다:

```bash
pip uninstall notebook jupyter ipywidgets -y
pip install notebook jupyter ipywidgets
```

---

위 단계를 모두 실행해도 문제가 해결되지 않는다면, 발생하는 에러 메시지와 실행 환경(윈도우 버전, Python 버전 등)을 공유해 주세요. 추가로 도와드리겠습니다.
